# Mikesell Charts Prototype
These charts aim to do at least two things:
1. Show the general shape of your strategy
2. Look at sub-range strategies

This prototype's aim is to give an idea of the parts and pieces needed to create these charts programmatically

## User Inputs
What parameters do we need from the user?

In [1]:
import ipywidgets as widgets
from IPython.display import display

In [2]:
street = widgets.Select(
    options=['Flop'],
    value='Flop',
    description='Street: ')

In [3]:
board = widgets.Select(
    options=['Ks7s6d'],
    value='Ks7s6d',
    description='Board: ')

In [4]:
preflop = widgets.Select(
    options=['3bp'],
    value='3bp',
    description='Preflop: ')

In [5]:
positions = widgets.Select(
    options=['BN v SB'],
    value='BN v SB',
    description='Positions: ')

In [6]:
vilains_action = widgets.Select(
    options=['OP'],
    value='OP',
    description="Hero's Position: ")

In [7]:
display(street, board, preflop, positions, vilains_action)

Notes:
- Start with only one flop spot
- Do with 100 flops
- Start with 3b or 4bps for simplicity

## Generate Sub-Ranges
Here we want to break down ranges into 'meaningful' sub-ranges.
- Digestible (not too many)
- Distinct (not overlapping)
- Understandable (why is solver doing that?)

For example, this graph helps the user see that there is a difference in top-set vs middle and bottom-set. It also show that you often will pot top-two but rarely bottom-two.

Sub-ranges will be generated programmatically rather than manually.

In [8]:
def trips():
    trips_list = []
    for i, rank in enumerate([x for i, x in enumerate(board.value) if i % 2 == 0]):
        temp_list = []
        temp_list.append(f'{rank}{rank}')
        
        if i == 0:
            ppt_exclude = ''
        elif i == 1:
            ppt_exclude = f'!{board.value[i*2-2]}{board.value[i*2-2]}'
        else:
            ppt_exclude = f'!({board.value[i*2-4]}{board.value[i*2-4]},{board.value[i*2-2]}{board.value[i*2-2]})'
            
        ppt = f'{rank}{rank}{ppt_exclude}'
        temp_list.append(ppt)

            
        trips_list.append(temp_list)
        
    return trips_list
    

In [9]:
def two_pair():
    A = board.value[0]
    B = board.value[2]
    C = board.value[4]
    
    exclude_base = f'!({A}{A},{B}{B},{C}{C}'
    top_two = [f'{A}{B}', f'{A}{B}{exclude_base})']
    top_bottom = [f'{A}{C}', f'{A}{C}{exclude_base},{A}{B})']
    bottom_two = [f'{B}{C}', f'{A}{B}{exclude_base},{A}{B},{A}{C})']
    
    return [top_two, top_bottom, bottom_two]
    
    
        

## Model the Output
The output can come in multiple formats.

I find that a bar chart when there are multiple bet sizes is nice. (Hero vs GTO)

And a line chart when comparing hero's strategy vs differing opponent stategies. (vMultiple Strats)

In [10]:
import matplotlib.pyplot as plt
import numpy as np

In [11]:
def create_plot(sub_range_labels, data):
    y_pos = np.arange(len(sub_range_labels))
    checks = np.array([x[0] for x in data])
    half = np.array([x[1] for x in data])
    pot = np.array([x[2] for x in data])

    p0 = plt.bar(y_pos, half, align='center', alpha=0.5)
    p1 = plt.bar(y_pos, pot, align='center', alpha=0.5, bottom=half)
    
    plt.xticks(y_pos, sub_range_labels)
    plt.xlabel('Sub-Range')
    plt.ylabel('%')
    plt.title('Betting Frequencies and Size vs ch/50/100')
    plt.legend(['Half', 'Pot'], loc='best')

    plt.show()

In [12]:
output = widgets.Output()

In [13]:
data = {
    "OP": [[50.7, 38.4, 10.9], [0.6, 99.4, 0], [7.4, 92.6, 0], [20.2, 34.6, 45.2], [38.0, 38.4, 23.5], [52.5, 41.1, 6.4]],
    "Adjust": [[48.1, 36.5, 15.4], [10, 90, 0], [24, 76, 0], [19.9, 33.2, 46.9], [42.6, 37.1, 20.3], [61.1, 31.5, 7.4]]
}

In [14]:
def on_post_button_click(b):
    with output:
        
        trip_list = trips()
        two_pair_list = two_pair()
        sub_range_labels = [x[0] for x in trip_list + two_pair_list]        
        create_plot(sub_range_labels, data[vilains_action.value])


In [15]:
post_button = widgets.Button(
    description='Hero vs GTO')

In [16]:
post_button.on_click(on_post_button_click)

In [17]:
display(post_button)

Button(description='Hero vs GTO', style=ButtonStyle())

In [18]:
def create_line_plot(sub_range_labels, data):
    y_pos = np.arange(len(sub_range_labels))
    half = np.array([x[1] for x in data[vilains_action.value]])
    pot = np.array([x[2] for x in data[vilains_action.value]])
    adj_half = np.array([x[1] for x in data['Adjust']])
    adj_pot = np.array([x[2] for x in data['Adjust']])
    
    p0 = plt.plot(y_pos, half+pot, color='C2')
    p1 = plt.plot(y_pos, adj_half+adj_pot, color='C3')
    
    plt.xticks(y_pos, sub_range_labels)
    plt.title('Betting Frequency')
    plt.xlabel('Sub-Range')
    plt.ylabel('%')
    plt.legend(['vs ch/50/100', 'vs ch/100'], loc='best')
    plt.show()   

In [19]:
def on_line_button_click(b):
    with output:
        
        trip_list = trips()
        two_pair_list = two_pair()
        sub_range_labels = [x[0] for x in trip_list + two_pair_list]        
        create_line_plot(sub_range_labels, data)


In [20]:
line_button = widgets.Button(
    description='vMultiple Strats')

In [21]:
line_button.on_click(on_line_button_click)

In [22]:
display(line_button, output)

Button(description='vMultiple Strats', style=ButtonStyle())

Output()